In [1]:
%load_ext autoreload
%autoreload 2
#from lib.multitasking_lib import *

In [2]:
from lib.multitasking_lib import *
cache_type ='backtest_web'

my_cache = cache_state('backtest_uni')

In [81]:
######################
# Initiate Backtest  #
######################
def store_algo(algo, algo_name="default"):
    redis_conn.hset("algos",algo_name, algo)
    algo_f = open("log/"+algo_name+".txt", "w")
    algo_f.write(algo)
    algo_f.close()
    
#my_cache.flushall()
def backtest_unitest(stocks, freq='1T',algo_name = 'haikin_1_new', fromDate='2019-4-8', toDate='2019-4-9', mode='full'):
    stocks = stocks
    sl = 1
    target = 1
    qty = 10
    backtest_msg={'stock':stocks,'sl':sl,'target':target,'qty':qty,'algo':algo_name,'fromDate':fromDate,'toDate':toDate,'freq':freq, 'mode':mode}

    # Step 4: Done is set to 0: Backtest is in progress, will be resetted by backtest job
    redis_conn.set('done'+cache_type,0)
    # Step 5: Send the msg to backtest thread to initiate the back test
    #pdebug(json.dumps(backtest_msg))
    redis_conn.publish('kite_simulator'+cache_type,json.dumps(backtest_msg))
    
##########################
# Unit Test              #
##########################

open('log/freedom.log','r+')
#log_file.close()

logger.setLevel(10)
loggerT.setLevel(21)
redis_conn.set('done'+cache_type,1)

try:
    if backtest_manager.job.is_alive():
        backtest_manager.job.terminate()
        time.sleep(0.1)
except:
    pass
finally:
    backtest_manager = threadManager(cache_type, ["kite_simulator","ohlc_tick_handler","order_handler"], [kite_simulator, ohlc_tick_handler, order_handler])

#redis_conn.publish('ohlc_tick_handler'+cache_type,'start')
print(backtest_manager.job.is_alive())
print(redis_conn.pubsub_channels())
print(jobs)


True
['order_handlerlive']
[]


2020-05-18 20:56:33,865 INFO:	Starting Handler: kite_simulator
2020-05-18 20:56:33,869 INFO:	Starting Handler: ohlc_tick_handler
2020-05-18 20:56:33,870 INFO:	Starting Handler: order_handler
2020-05-18 20:56:33,898 DEBUG:	kite_simulator: 1
2020-05-18 20:56:33,934 DEBUG:	ohlc_tick_handler(backtest_web) - INIT: 1
2020-05-18 20:56:33,945 DEBUG:	order_handler(backtest_web): 1
2020-05-18 20:56:33,954 ERROR:	kite_simulator: Invalid msg: 1
2020-05-18 20:56:34,016 ERROR:	Error in order handler
2020-05-18 20:56:35,084 DEBUG:	kite_simulator: {"stock": ["TCS"], "sl": 1, "target": 1, "qty": 10, "algo": "haikin_1_new", "fromDate": "2019-4-8", "toDate": "2019-4-9", "freq": "1T", "mode": "quick"}
2020-05-18 20:56:35,097 DEBUG:	Trade_init: TCS
2020-05-18 20:56:35,109 INFO:	Reset Cache for: TCSbacktest_web
2020-05-18 20:56:35,131 INFO:	TCSbacktest_web=>{'stock': 'TCS', 'qty': '0', 'SL %': '0.0', 'TP %': '0.0', 'amount': '0', 'price': '0.0', 'P&L': '0.0', 'P&L %': '0.0', 'Total P&L': '0.0', 'Total P&L %

<class 'pandas.core.series.Series'>


2020-05-18 20:56:36,739 INFO:	                        sell      buy
date                                 
2019-04-05 15:07:00  2047.00      NaN
2019-04-05 15:09:00      NaN  2047.60
2019-04-05 15:12:00  2046.55      NaN
2019-04-05 15:14:00      NaN  2048.55
2019-04-05 15:15:00  2047.00      NaN
2019-04-05 15:18:00      NaN  2048.50
2019-04-05 15:24:00  2049.30      NaN
2019-04-05 15:25:00      NaN  2050.85
2019-04-05 15:26:00  2049.90      NaN
2019-04-05 15:29:00      NaN  2051.00


In [82]:
backtest_unitest(['TCS'], mode='quick')

In [79]:
pd.read_json(my_cache.get('buy_df'), orient='index')[0].values

array([False, False, False, ..., False, False,  True])

In [89]:
pd.read_json(my_cache.get('TCSbacktest_webTrade'))

,sell,buy
2019-04-03 09:19:00,NaN,2076.00
2019-04-03 09:20:00,2072.0,NaN
2019-04-03 09:22:00,NaN,2071.25
2019-04-03 09:25:00,2071.8,NaN
2019-04-03 09:27:00,NaN,2072.10
...,...,...
2019-04-05 15:18:00,NaN,2048.50
2019-04-05 15:24:00,2049.3,NaN
2019-04-05 15:25:00,NaN,2050.85
2019-04-05 15:26:00,2049.9,NaN


In [80]:
################################
###         Stop Trade       ###
################################
pinfo(cache_type)
my_cache.xtrim('msgBufferQueue'+cache_type,0,False)
my_cache.delete('msgBufferQueue'+cache_type)
my_cache.set('TCSbacktest_uniTICK',pd.DataFrame().to_json())

if backtest_manager.job.is_alive():
    backtest_manager.job.terminate()
time.sleep(1)
print(backtest_manager.job.is_alive())
print(redis_conn.pubsub_channels())
my_cache.set('last_id_msg',0)
jobs
jobs = []

2020-05-18 20:56:30,679 INFO:	backtest_web


False
['order_handlerlive']


In [21]:
redis_conn.pubsub_channels()

['ohlc_tick_handlerbacktest_web',
 'kite_simulatorbacktest_web',
 'order_handlerlive']

In [22]:
backtest_unitest(['TCS'])

In [23]:
backtest_unitest(['WIPRO','TCS'])

In [24]:
from lib.charting_lib import *
cache_type = 'backtest_uni'
chart_type ='haikin'

symbol='TCS'
my_cache = cache_state(cache_type)

#######  Hack for freq  ############
my_cache.setValue('TCS','freq','1T')
#####################################
dfohlc = my_cache.getOHLC(symbol)
print(dfohlc.tail())
trade_df = pd.read_json(redis_conn.get(symbol+cache_type+'Trade'), orient='columns')

render_charts(dfohlc, trade_df, symbol, chart_type)

Empty DataFrame
Columns: []
Index: []


''

In [ ]:
cache_state('backtest_uni').getValue()

## Live Trade Debugging

In [185]:
live = threadManager('live', ["ohlc_tick_handler"], [ohlc_tick_handler])


INFO:	Starting Handler: ohlc_tick_handler
DEBUG:	ohlc_tick_handler(live) - INIT: 1
TRADE: S: EN:  : 2020-05-13 15:10 : WIPRO -> 189.3
TRADE: B: EX:  : 2020-05-13 15:16 : WIPRO -> 189.4
TRADE: S: EN:  : 2020-05-13 15:19 : WIPRO -> 190.0
TRADE: B: EX:  : 2020-05-13 15:20 : WIPRO -> 190.05
TRADE: S: EN:  : 2020-05-13 15:23 : WIPRO -> 189.95
TRADE: B: EX:  : 2020-05-13 15:24 : WIPRO -> 190.05
TRADE: S: EN:  : 2020-05-13 15:27 : WIPRO -> 189.9
TRADE: B: EX:  : 2020-05-13 16:18 : WIPRO -> 189.7
ERROR:	Could not read data from msgBufferQueue


In [116]:
live_cache.setValue('WIPRO','mode','paper')

0

In [184]:
live.job.terminate()
live_cache.set('WIPROliveTICK', pd.DataFrame().to_json(orient='columns'))
live_cache.setValue('WIPRO','last_processed',0)
live_cache.getValue('WIPRO')

,stock,qty,SL %,TP %,amount,price,P&L,P&L %,Total P&L,Total P&L %,...,ltp %,tp,high,last_processed,state,mode,algo,freq,hdf_freq,order_id
0,WIPRO,1,1,1,189.7,0,0.20000000000001705,0.10531858873091997,-0.5000000000000568,-0.2635740643121017,...,0.0,0,189.7,0,SCANNING,paper,haikin_1_new,1T,minute,-1


In [73]:
#live_cache.getOHLC('WIPRO') 
print(live_cache.xrange('msgBufferQueuelive')[-2:])
print(datetime.fromtimestamp(int(live_cache.xrange('msgBufferQueuelive')[-1][0].split('-')[0] )/1000))


[('1589365238353-0', {'data': '[{"tradable": true, "mode": "ltp", "instrument_token": 969473, "last_price": 189.7}]'}), ('1589365249733-0', {'data': '[{"tradable": true, "mode": "ltp", "instrument_token": 969473, "last_price": 189.7}]'})]
2020-05-13 15:50:49.733000


In [11]:
live_cache.getOHLC('WIPRO').tail()

NameError: name 'live_cache' is not defined

## Quick Backtest

In [ ]:
###########################################################
###                  Quick Backtest                     ###
###########################################################
from talib import MACD, MACDEXT, RSI, BBANDS, MACD, AROON, STOCHF, ATR, OBV, ADOSC, MINUS_DI, PLUS_DI, ADX, EMA, SMA
from talib import LINEARREG, BETA, LINEARREG_INTERCEPT, LINEARREG_SLOPE, STDDEV, TSF, ADOSC, VAR, ROC, MIN, MAX, MINMAX

ohlc_get = lambda df, key: df.iloc[-1][key]
def HAIKINASI(ohlc_data_df):
    REF = lambda key, i: ohlc_get(ohlc_data_df.shift(i), key)
    
    OPEN  = ohlc_data_df['open']
    HIGH  = ohlc_data_df['high']
    LOW   = ohlc_data_df['low']
    CLOSE = ohlc_data_df['close']
    
    haOPEN  = (OPEN.shift(1) + CLOSE.shift(1))/2
    haHIGH  = pd.DataFrame([HIGH,OPEN,CLOSE]).max(axis = 0, skipna = True)
    haLOW   = pd.DataFrame([LOW,OPEN,CLOSE]).min(axis = 0, skipna = True)
    haCLOSE = (OPEN+HIGH+LOW+CLOSE)/4
    
    return (haOPEN, haHIGH, haLOW, haCLOSE)


################ Load Data ###################
from lib.charting_lib import *
cache_type = 'backtest_uni'
symbol='TCS'
my_cache = cache_state(cache_type)

###  Set Freq for Algo
my_cache.setValue('TCS','freq','1T')
dfohlc = my_cache.getOHLC(symbol)
#print(dfohlc.head())

### Load Data
ohlc_data_temp = dfohlc
pd.set_option('display.max_rows', None)
################ Initialize Helper Functions ###################
trade_df1 = pd.DataFrame()
OPEN = ohlc_data_temp['open']
CLOSE = ohlc_data_temp['close']
HIGH = ohlc_data_temp['high']
LOW = ohlc_data_temp['low']
#VOLUME = ohlc_data_temp['volume']

(haOPEN, haHIGH, haLOW, haCLOSE) = HAIKINASI(ohlc_data_temp)

TIME = ohlc_data_temp.index[-1].minute+ohlc_data_temp.index[-1].hour*60

#REF = lambda df, i: df.iloc[-i-1]
REF = lambda df, i: df.shift(i)
TREND_UP = lambda a,b: ROC(a, b) >= 0.1
TREND_DOWN = lambda a,b: ROC(a, b) <= -0.1
CROSSOVER = lambda a, b: (REF(a,1)<=REF(b,1)) & (a > b)


def BUY(CLOSE, x, trade_df1):
    #global trade_df1
    tmp_df = pd.DataFrame()
    tmp_df["buy"] = CLOSE[x]
    trade_df1 = trade_df1.append(tmp_df)
    return trade_df1

def SELL(CLOSE, x, trade_df1):
    #global trade_df1
    tmp_df = pd.DataFrame()
    tmp_df["sell"] = CLOSE[x]
    trade_df1 = trade_df1.append(tmp_df)
    return trade_df1
    
################# Write Algo For Backtest Here ###################

#sell = (REF(haOPEN, 0) > REF(haCLOSE,0)) & (REF(haOPEN, 1) > REF(haCLOSE,1))
#print(trade_df1)
sell = (REF(haOPEN, 0) > REF(haCLOSE,0)) & (REF(haOPEN, 1) < REF(haCLOSE,1))
buy = (REF(haOPEN, 0) < REF(haCLOSE,0)) & (REF(haOPEN, 1) > REF(haCLOSE,1))

logic = buy | sell

summary_df = trade_df1.copy(deep=True)
#summary_df['haopen'] = haOPEN[ logic ]
#summary_df['haclose'] = haCLOSE[ logic ]
summary_df['roc'] = ROC(CLOSE, 30)[logic]

#print(summary_df.sort_index())
#print(summary_df.sort_index().tail(10))
#print(ROC(CLOSE,3))
#################### Render Charts ###############################

trade_df1 = SELL(CLOSE, sell, trade_df1)
trade_df1 = BUY(CLOSE, buy, trade_df1)

#print(trade_df1.sort_index())
my_cache.setCache(symbol+cache_type+'Trade',trade_df1)
#my_cache.pushTrade(symbol, trade_df1)

chart_type ='haikin'
trade_df = pd.read_json(redis_conn.get(symbol+cache_type+'Trade'), orient='columns')

render_charts(ohlc_data_temp, trade_df, symbol, chart_type)

In [ ]:
cache_type = 'backtest_web'
symbol='TCS'
my_cache = cache_state(cache_type)
my_cache.getTrades(symbol)

my_cache.get('TCSbacktest_webTrades')

In [ ]:
my_cache.getValue()

In [ ]:
temp_file = pd.HDFStore("data/kite_cache.h5", mode="r")
temp_file.keys()

In [9]:
live_cache.hget('algos','SUHAN') is not None

False

## Redis Console

In [3]:
from lib.multitasking_lib import *
#from lib.data_model_lib import *
import json
web_cache = cache_state('backtest_web')
live_cache = cache_state('live')
uni_cache = cache_state('backtest_uni')

pd.set_option('display.max_rows', None)

In [4]:
live_cache.pubsub_channels()

['ohlc_tick_handlerlive',
 'ohlc_tick_handlerbacktest_web',
 'kite_simulatorbacktest_web',
 'kite_ticker_handlerlive',
 'order_handlerbacktest_web',
 'order_handlerlive']

In [5]:
values = ['TCS','WIPRO','CIPLA','AXISBANK']
df_cache = live_cache.getValue()
cache_keys =  live_cache.getKeys()

if values is not None:
    for value in values:
        if value not in live_cache.getKeys():
            print(value)

TCS
CIPLA


In [6]:
live_cache.remove()

In [5]:
df = live_cache.getValue()
df

,stock,qty,SL %,TP %,amount,price,P&L,P&L %,Total P&L,Total P&L %,...,ltp %,tp,high,last_processed,state,mode,algo,freq,hdf_freq,order_id
0,CIPLA,1,0.4,1,571.0,0,0.39999999999997726,0.07010164738870965,-2903.3999999999996,-508.4763572679509,...,0.0,0,571.8,1589535938.388,SCANNING,paper,haikin_1_new,1T,day,200515001166774
1,WIPRO,1,0.4,1,184.05,184.05,0,0,182.6,99.21217060581363,...,0.0,182.20950000000002,185.15,1589535938.388,SHORT,paper,haikin_1_new,1T,minute,200514002588219
2,BAJAJ-AUTO,1,0.4,1,0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1589611349.692351,INIT,paper,haikin_1_new,1T,day,0
3,AXISBANK,1,0.4,1,0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1589634820.029116,INIT,paper,haikin_1_new,1T,day,0
4,INFRATEL,1,0.4,1,202.55,202.55,0,0,1.1000000000000227,0.5430757837571082,...,0.0,204.5755,206.15,1589535942.01,LONG,live,haikin_1_new,1T,day,200515003518804


In [ ]:
live_cache.getOHLC('TCS')

In [ ]:
trades_df = live_cache.getTrades('TCS')
trades_df

In [ ]:
(total_profit, max_loss, max_profit, total_win, total_loss, max_winning_streak, max_loosing_streak, trade_log_df) = trade_analysis_raw(trades_df)

In [ ]:
import pandas as pd
import plotly
import plotly.graph_objs as go
from datetime import datetime as dt
from datetime import timedelta
from plotly.subplots import make_subplots
import plotly.graph_objects as go

from lib.multitasking_lib import *
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

### GetData

In [ ]:
logger.setLevel(logging.DEBUG)
stock = 'WIPRO'
fromDate = '2019-4-22'
toDate  = '2019-4-24'
freq = 'minute'
mydate = fromDate
exchange = 'NSE'
ohlc_data = {}
ohlc_data[stock] = getData(stock, fromDate, toDate, exchange, freq, False, stock)

In [ ]:
ohlc_data[stock].head()

# Kite Connect

In [6]:
from kiteconnect import KiteConnect
kite_api_key = 'b2w0sfnr1zr92nxm'
live_cache.set('KiteAPIKey',kite_api_key)
access_token = live_cache.get('access_token')
print(access_token)
print(live_cache.get('KiteAPIKey'))

kite = KiteConnect(api_key=kite_api_key)
kite.set_access_token(access_token)

4GfO20yFuBCzEo81yg07HcV4u1y4fyKW
b2w0sfnr1zr92nxm


In [7]:
kite.access_token

'4GfO20yFuBCzEo81yg07HcV4u1y4fyKW'

In [8]:
kite.profile()

{'user_id': 'YM1755',
 'user_type': 'individual',
 'email': 'saswati.suhan@gmail.com',
 'user_name': 'Saswati Saha',
 'user_shortname': 'Saswati',
 'broker': 'ZERODHA',
 'exchanges': ['CDS', 'MF', 'BFO', 'MCX', 'NSE', 'BSE', 'NFO'],
 'products': ['CNC', 'NRML', 'MIS', 'BO', 'CO'],
 'order_types': ['MARKET', 'LIMIT', 'SL', 'SL-M'],
 'avatar_url': '',
 'meta': {'demat_consent': 'physical'}}

In [ ]:
cancel_order('CIPLA')

In [ ]:
orders_df = pd.DataFrame(kite.orders())
orders_df[['order_id','status','tradingsymbol','transaction_type','quantity','tag','price']]

In [6]:
live_cache.getValue()

,stock,qty,SL %,TP %,amount,price,P&L,P&L %,Total P&L,Total P&L %,...,ltp %,tp,high,last_processed,state,mode,algo,freq,hdf_freq,order_id
0,CIPLA,1,1,1,572.85,572.85,0,0,-2.0,-0.3491315353059265,...,0.0,567.1215,576.0,2020-05-14 09:31,SHORT,live,haikin_1_new,1T,minute,0
1,WIPRO,1,1,1,183.6,183.6,0,0,-0.15000000000000568,-0.08169934640523187,...,0.0,181.76399999999998,184.35,2020-05-14 09:31,SHORT,paper,haikin_1_new,1T,minute,0


In [ ]:
position_df = pd.DataFrame(kite.positions()['net'])
position_df

In [ ]:
pd.DataFrame(kite.positions()['day'])

In [11]:
orders_df = pd.DataFrame()
for msg in live_cache.xrange('notificationQueuelivenew'):
    tmp_df = pd.DataFrame.from_dict(json.loads(msg[1]['data']), orient='index').transpose()
    orders_df = orders_df.append(tmp_df)
    #print(msg)
orders_df[['order_id', 'status', 'tradingsymbol','order_type','transaction_type','quantity', 'average_price']]

,order_id,status,tradingsymbol,order_type,transaction_type,quantity,average_price
0,200512000988113,OPEN,WIPRO,MARKET,SELL,2,0
0,200512000988113,COMPLETE,WIPRO,MARKET,SELL,2,185.05
0,200512000820643,COMPLETE,WIPRO,LIMIT,SELL,1,185.7
0,200512001243114,OPEN,WIPRO,LIMIT,SELL,1,0
0,200512001243114,COMPLETE,WIPRO,LIMIT,SELL,1,186.55
0,200512001260328,COMPLETE,WIPRO,MARKET,BUY,2,186.35
0,200512001260328,OPEN,WIPRO,MARKET,BUY,2,0
0,200512001267483,OPEN,CIPLA,LIMIT,SELL,1,0
0,200512001267483,COMPLETE,CIPLA,LIMIT,SELL,1,582.3
0,200512001288599,COMPLETE,WIPRO,LIMIT,BUY,1,186.3


In [15]:
msgs_q = live_cache.xread({'notificationQueuelivenew':'0'}, block=2000, count=5000)

In [32]:
for msg in msgs_q[0][1]:
    print(msg[0])
    #print(msg[1]['data'])
    data = json.loads(msg[1]['data'])
    order_id = data['order_id']
    status = data['status']
    symbol = data['tradingsymbol']
    symbol_state = live_cache.getValue(symbol,'state')
    print("{} => {}".format(status, symbol_state))
    break

1589257940467-0
OPEN=>SCANNING


In [10]:
orders_df.columns

Index(['placed_by', 'order_id', 'exchange_order_id', 'parent_order_id',
       'status', 'status_message', 'status_message_raw', 'order_timestamp',
       'exchange_update_timestamp', 'exchange_timestamp', 'variety',
       'exchange', 'tradingsymbol', 'instrument_token', 'order_type',
       'transaction_type', 'validity', 'product', 'quantity',
       'disclosed_quantity', 'price', 'trigger_price', 'average_price',
       'filled_quantity', 'pending_quantity', 'cancelled_quantity',
       'market_protection', 'meta_raw', 'meta', 'tag', 'guid', 'account_id',
       'unfilled_quantity', 'app_id', 'checksum', 'tags'],
      dtype='object')

In [ ]:
kite.margins()

#### Starting Live Trade for a stock

In [ ]:
symbol = 'INFY'
live_cache.add(symbol)
token = int(live_cache.hmget('eq_token',symbol)[0])
live_cache.publish('live_trade_handlerlive', json.dumps({'cmd':'add','value':[token],'mode':'ltp'}))

live_cache.setValue(symbol,'qty','1')
live_cache.setValue(symbol,'SL %','0.4')
live_cache.setValue(symbol,'TP %','1')
live_cache.setValue(symbol,'algo','haikin_1_new')
live_cache.setValue(symbol,'freq','1T')
live_cache.setValue(symbol,'last_processed',datetime.now().timestamp())
live_cache.setValue(symbol,'mode','paper')

In [163]:
symbol = 'WIPRO'
live_cache.setValue(symbol,'last_processed',datetime.now().timestamp())

0

In [161]:
datetime.now().timestamp()

1589454314.433045

In [ ]:
symbol = 'INFY'
live_cache.setValue(symbol,'mode','PAUSE')

In [164]:
live_cache.getValue(symbol)

,stock,qty,SL %,TP %,amount,price,P&L,P&L %,Total P&L,Total P&L %,...,ltp %,tp,high,last_processed,state,mode,algo,freq,hdf_freq,order_id
0,WIPRO,1,1,1,186.7,0,-0.05000000000001137,-0.02677376171352684,182.0,97.48259239421533,...,0.0,0,187.35,1589454443.512455,SCANNING,paper,haikin_1_new,1T,minute,200514002588219


### Kite Ticker Handler Manipulation

In [ ]:
#live_cache.publish('live_trade_handlerlive','profile')
live_cache.publish('live_trade_handlerlive','status')

In [21]:
live_cache.publish('kite_ticker_handlerlive','INIT')

1

In [20]:
live_cache.publish('kite_ticker_handlerlive','CLOSE')

1

In [16]:
live_cache.publish('kite_ticker_handlerlive','START')

1

In [17]:
live_cache.publish('kite_ticker_handlerlive','profile')

1

In [ ]:
live_cache.smembers('live_token')

In [78]:
values = list(map(int,live_cache.smembers('ticker_list')))
values

[315393, 969473, 134657]

In [79]:
live_cache.publish('kite_ticker_handlerlive',json.dumps({'cmd':'add','value':values,'mode':'ltp'}))

1

In [68]:
live_cache.publish('kite_ticker_handlerlive',json.dumps({'cmd':'remove','value':[1510401],'mode':'quote'}))


1

In [ ]:
pd.DataFrame(kite.orders())

#### TickerList Manipulaitons

In [ ]:
live_cache.sadd('ticker_list','2953217')
live_cache.sadd('ticker_list','969473')

live_cache.sadd('ticker_list','1510401')

In [74]:
values = list(map(int,live_cache.smembers('ticker_list')))
print(values)
live_cache.hmget('eq_token', values)

[1510401, 969473, 7458561]


['AXISBANK', 'WIPRO', 'INFRATEL']

In [20]:
live_cache.srem('ticker_list',1510401)

1

In [ ]:
live_cache.sismember('ticker_list',4268801)

In [21]:
live_cache.delete('ticker_list')

1

In [72]:
live_cache.getValue()

,stock,qty,SL %,TP %,amount,price,P&L,P&L %,Total P&L,Total P&L %,...,ltp %,tp,high,last_processed,state,mode,algo,freq,hdf_freq,order_id
0,INFRATEL,1,0.4,1,0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,209.65,1589489283.667,SCANNING,paper,haikin_1_new,1T,day,0
1,WIPRO,1,1,1,186.0,186.0,0,0,182.0,97.84946236559139,...,0.0,184.14,186.5,1589489283.667,SCANNING,paper,haikin_1_new,1T,minute,200514002588219


### Get Kite Ticker Status

In [8]:
## Get Status of Kite
live_cache.get('Kite_Status')

'connected'

### OrderHandler Monitor and Manipulation

In [47]:
# Pause Ordering
live_cache.publish('order_handlerlive', 'pause')

1

In [59]:
# Resume Ordering
live_cache.publish('order_handlerlive', 'resume')

1

In [44]:
# Buy
live_cache.publish('order_handlerlive',json.dumps({'cmd':'buy','symbol':'WIPRO','price':180,'qty':1}))

1

In [69]:
# Sell
live_cache.publish('order_handlerlive',json.dumps({'cmd':'sell','symbol':'WIPRO','price':190,'qty':1}))

1

In [ ]:
# Cancel
live_cache.publish('order_handlerlive',json.dumps({'cmd':'cancel','symbol':'WIPRO'}))

In [ ]:
live_cache.publish('order_handlerlive',json.dumps({'cmd':'getOrders'}))

In [33]:
## Cancel All the orders
live_cache.publish('order_handlerlive',json.dumps({'cmd':'cancelAll'}))

1

In [ ]:
#from lib.kite_wrapper_lib import *
cancel_order(kite,['CIPLA'])

In [ ]:
buy_limit('WIPRO',188.5,1)

In [ ]:
sell_limit('WIPRO',188.7,1)

In [11]:
live_cache.getOHLC('WIPRO').tail(10)

""


In [ ]:
kite.cancel_order(kite.VARIETY_REGULAR, 200511002924441, parent_order_id=None)

In [ ]:
kite.convert_position(kite.EXCHANGE_NSE, 'WIPRO', transaction_type, position_type, quantity, old_product, new_product)


In [ ]:
kite.modify_order(kite.VARIETY_REGULAR, 200511003019322, parent_order_id=None, quantity=None, price=187.4, order_type=None, trigger_price=None, validity=None, disclosed_quantity=None)


In [ ]:
msg = {'data': '[{"tradable": true, "mode": "quote", "instrument_token": 969473, "last_price": 184.7, "last_quantity": 5, "average_price": 187.07, "volume": 4393242, "buy_quantity": 0, "sell_quantity": 0, "ohlc": {"open": 185.0, "high": 189.95, "low": 184.1, "close": 184.0}, "change": 0.38043478260868946}, {"tradable": true, "mode": "quote", "instrument_token": 2953217, "last_price": 1935.3, "last_quantity": 25, "average_price": 1939.47, "volume": 2449395, "buy_quantity": 0, "sell_quantity": 0, "ohlc": {"open": 1909.0, "high": 1951.4, "low": 1909.0, "close": 1893.4}, "change": 2.2129502482306886}]'}

In [ ]:
msg['data']

In [ ]:
val = json.loads(msg['data'])

##### Reset state for tradejob

In [ ]:
live_cache.setValue('WIPRO','state','SCANNING')

### Live Cache State Monitor and manipulation

In [55]:
live_cache.getValue()

,stock,qty,SL %,TP %,amount,price,P&L,P&L %,Total P&L,Total P&L %,...,ltp %,tp,high,last_processed,state,mode,algo,freq,hdf_freq,order_id
0,BPCL,1,0.4,1,0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,309.7,1589488457.915,INIT,paper,haikin_1_new,1T,day,0
1,WIPRO,1,1,1,186.0,186.0,0,0,182.0,97.84946236559139,...,0.0,184.14,186.5,1589488457.915,INIT,paper,haikin_1_new,1T,minute,200514002588219
2,AXISBANK,1,0.4,1,0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,401.95,1589488457.915,INIT,paper,haikin_1_new,1T,day,0


In [11]:
#Modify cache table
stock_key = 'CIPLA'
#live_cache.add(stock_key, True) #TODO: While adding stock to table use True
live_cache.setValue(stock_key, 'algo', 'haikin_1_new')
live_cache.setValue(stock_key, 'freq', '1T')
live_cache.setValue(stock_key, 'qty', 1)
live_cache.setValue(stock_key, 'SL %', 1)
live_cache.setValue(stock_key, 'TP %', 1)
live_cache.setValue(stock_key, 'P&L', 0)
live_cache.setValue(stock_key, 'Total P&L', 0)
live_cache.setValue(stock_key, 'price', 0)
live_cache.setValue(stock_key, 'hdf_freq', 'minute')
live_cache.getValue()

2020-05-14 09:15:22,066 INFO:	Reset Cache for: CIPLAlive
2020-05-14 09:15:22,084 INFO:	CIPLAlive=>{'stock': 'CIPLA', 'qty': '0', 'SL %': '0.0', 'TP %': '0.0', 'amount': '0', 'price': '0.0', 'P&L': '0.0', 'P&L %': '0.0', 'Total P&L': '0.0', 'Total P&L %': '0.0', 'low': '0.0', 'sl': '0.0', 'ltp': '0.0', 'ltp %': '0.0', 'tp': '0.0', 'high': '0.0', 'last_processed': '1999-01-01', 'state': 'INIT', 'mode': 'PAUSE', 'algo': '', 'freq': '1D', 'hdf_freq': 'day', 'order_id': '0'}


,stock,qty,SL %,TP %,amount,price,P&L,P&L %,Total P&L,Total P&L %,...,ltp %,tp,high,last_processed,state,mode,algo,freq,hdf_freq,order_id
0,CIPLA,1,1,1,0,0,0,0.0,0,0.0,...,0.0,0.0,0.0,1999-01-01,INIT,PAUSE,haikin_1_new,1T,minute,0
1,WIPRO,1,1,1,0,0,0,0.0,0,0.0,...,0.0,0.0,0.0,1999-01-01,INIT,PAUSE,haikin_1_new,1T,minute,0


In [58]:
live_cache.setValue('WIPRO', 'mode', 'paper')

0

In [115]:
#live_cache.remove('AXISBANK')
live_cache.getValue()

,stock,qty,SL %,TP %,amount,price,P&L,P&L %,Total P&L,Total P&L %,...,ltp %,tp,high,last_processed,state,mode,algo,freq,hdf_freq,order_id
0,CIPLA,1,0.4,1,581.75,0,-581.75,-inf,-2908.35,-499.9312419424151,...,0.0,0,582.95,1589517856.188,SCANNING,paper,haikin_1_new,1T,day,200515001166774
1,WIPRO,1,0.4,1,183.9,183.9,0,0,182.0,98.96682979880369,...,0.0,185.739,184.8,1589517887.406,SCANNING,paper,haikin_1_new,1T,minute,200514002588219
2,INFRATEL,1,0.4,1,203.05,203.05,0,0,0.0,0.0,...,0.0,201.01950000000002,204.5,1589517879.133,SCANNING,live,haikin_1_new,1T,day,200515001219797


In [9]:
live_cache.remove()

#### Get token for Stock symbol

In [ ]:
live_cache.hmget('eq_token',['CIPLA','INFY'])

In [ ]:
live_cache.hmget('eq_token',['TCS','WIPRO','AXISBANK'])

#### Peek into OHLC buffer

In [9]:
live_cache.getOHLC('INFRATEL').tail(5)

,close,high,low,open
2020-05-15 15:15:00,202.30,202.60,202.25,202.50
2020-05-15 15:16:00,202.35,202.55,202.25,202.35
2020-05-18 09:19:00,207.15,207.15,206.55,207.15
2020-05-18 09:20:00,207.10,207.50,206.65,207.45
2020-05-18 09:21:00,206.90,207.25,206.90,207.25


#### msgBufferQueue Monitoring

In [11]:
peek_msg_queue = live_cache.xrange('msgBufferQueuelive')[-1]
print(peek_msg_queue)
datetime.fromtimestamp(int(peek_msg_queue[0].split('-')[0] )/1000)

('1589773891139-0', {'data': '[{"tradable": true, "mode": "ltp", "instrument_token": 1510401, "last_price": 379.8}, {"tradable": true, "mode": "ltp", "instrument_token": 7458561, "last_price": 206.65}, {"tradable": true, "mode": "ltp", "instrument_token": 177665, "last_price": 584.0}, {"tradable": true, "mode": "ltp", "instrument_token": 969473, "last_price": 182.6}, {"tradable": true, "mode": "ltp", "instrument_token": 4267265, "last_price": 2645.7}]'})


datetime.datetime(2020, 5, 18, 9, 21, 31, 139000)

In [49]:
datetime.fromtimestamp(1589428580968/1000)

2020-05-14 15:02:10,275 DEBUG:	freedom.log
2020-05-14 15:02:10,275 DEBUG:	freedom.log


datetime.datetime(2020, 5, 14, 9, 26, 20, 968000)

In [168]:
live_cache.getOHLC('INFRATEL').tail()

,close,high,low,open
2020-05-15 14:53:00,202.45,202.5,202.20,202.20
2020-05-15 14:54:00,202.70,202.7,202.40,202.40
2020-05-15 15:04:00,202.45,202.5,202.35,202.35
2020-05-15 15:05:00,202.15,202.5,202.10,202.45
2020-05-15 15:06:00,202.15,202.2,201.70,202.20


##### Monitor: OHLC Tick Handler

In [ ]:
val = live_cache.get('last_id_msg')
print(val)
datetime.fromtimestamp(int(val.split('-')[0] )/1000)

In [62]:
# Timestamp to ID
timestamp_to_id = lambda x:str(int(float(x)*1000))+'-'+str(0)
id_val = timestamp_to_id(live_cache.getValue('WIPRO','last_processed'))
print(id_val)
live_cache.xread({'msgBufferQueuelive':id_val}, count=1)

1589513840364-0


[['msgBufferQueuelive',
  [('1589513890167-0',
    {'data': '[{"tradable": true, "mode": "ltp", "instrument_token": 969473, "last_price": 186.0}, {"tradable": true, "mode": "ltp", "instrument_token": 7458561, "last_price": 205.95}]'})]]]

In [13]:
# Peek into the last msg processed by tick handler
print(live_cache.getValue('WIPRO','last_processed'))
datetime.fromtimestamp(float(live_cache.getValue('WIPRO','last_processed')))

1589773904.994


datetime.datetime(2020, 5, 18, 9, 21, 44, 994000)

In [15]:
## Ticker job monitoring
live_cache.get('tick_count')

'6895'

In [175]:
datetime.fromtimestamp(1589535456664/1000)

datetime.datetime(2020, 5, 15, 15, 7, 36, 664000)

In [170]:
msgBufferDump = live_cache.xrange('msgBufferQueuelive')

In [171]:
fp = open('log/ticker_log.log',mode='w')

In [174]:
fp.write(json.dumps(msgBufferDump))

4021969

### Update Redis with Instrument list

##### Download fresh data and store in cache

In [ ]:
instruments_df = pd.DataFrame(kite.instruments())

In [ ]:
instruments_df.to_hdf('data/instruments.h5',key='instruments')

In [ ]:
instruments_df.columns

In [ ]:
eq_nse = instruments_df.loc[(instruments_df['exchange']=='NSE') & (instruments_df['segment']=='NSE')  & (instruments_df['instrument_type']=='EQ'), ['instrument_token','tradingsymbol']]

In [ ]:
eq_nse.loc[eq_nse['tradingsymbol'].isin(['TCS','WIPRO']),'instrument_token'].values

##### Use cache

In [ ]:
instruments_df = pd.read_hdf('data/instruments.h5',key='instruments')

In [ ]:
instruments_df

In [ ]:
eq_nse = instruments_df.loc[(instruments_df['exchange']=='NSE') & (instruments_df['segment']=='NSE')  & (instruments_df['instrument_type']=='EQ'), ['instrument_token','tradingsymbol']]

In [ ]:
eq_nse.loc[eq_nse['tradingsymbol'].isin(['TCS','WIPRO']),:].values

eq_nse.set_index('instrument_token')

In [ ]:
live_cache.delete('eq_token')

In [ ]:
eq_nse.head().to_dict(orient='records')

In [ ]:
eq_nse.head().set_index('tradingsymbol').transpose().to_dict(orient='records')

In [ ]:
eq_nse.head().set_index('instrument_token').transpose().to_dict(orient='records')[0]

##### Update redis with tokens

In [ ]:
live_cache.hmset('eq_token',eq_nse.set_index('instrument_token').transpose().to_dict(orient='records')[0])
live_cache.hmset('eq_token',eq_nse.set_index('tradingsymbol').transpose().to_dict(orient='records')[0])

In [ ]:
live_cache.hmget('eq_token',['TCS','WIPRO'])

In [ ]:
live_cache.hgetall('eq_token')

In [ ]:
live_cache.hmget('eq_token',[2953217, 969473])